In [36]:
import pdb
import os
import numpy as np
import pandas as pd
import pylab
import pylab as plt
import seaborn
from utils import find_nearest
from utils import find_nearest_index
from GameDayFunctions.fangraphs_projection import Projection
from GameDayFunctions.draft import Draft

%matplotlib inline

In [4]:
ZiPS = Projection()
ZiPS.precompute_statlines()

In [5]:
Steamer = Projection(model = 'Steamer')
Steamer.precompute_statlines()

In [6]:
Fans = Projection(model = 'Fans')
Fans.precompute_statlines()

In [20]:
ZiPS.statline['batters'][ZiPS.statline['batters']['Position'] == 'OF'].head()

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,AB,Position,Rank,Name,Drafted
playerid,,,,,,,,,,,,,,,
10155,105,76,26,4,38,92,25,118,0.291,1.023,495,OF,1,Mike Trout,False
20123,106,94,31,4,36,112,8,98,0.296,0.959,558,OF,14,Juan Soto,False
11579,104,74,30,1,35,109,14,119,0.271,0.944,516,OF,10,Bryce Harper,False
6184,88,83,32,2,36,106,4,59,0.289,0.922,529,OF,3,J.D. Martinez,False
15640,86,65,20,1,35,90,7,84,0.260,0.907,466,OF,4,Aaron Judge,False


In [22]:
Steamer.statline['batters'][Steamer.statline['batters']['Position'] == 'OF'].head()

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,AB,Position,Rank,Name,Drafted
playerid,,,,,,,,,,,,,,,
10155,118,81,31,4,44,109,21,118,0.306,1.074,522,OF,1,Mike Trout,False
6184,106,95,35,2,44,121,4,67,0.304,0.977,580,OF,3,J.D. Martinez,False
13611,112,105,42,4,32,102,26,77,0.305,0.936,598,OF,2,Mookie Betts,False
11579,93,70,28,2,35,95,10,108,0.264,0.924,511,OF,10,Bryce Harper,False
15640,103,75,26,2,40,99,7,95,0.263,0.914,543,OF,4,Aaron Judge,False


In [21]:
Fans.statline['batters'][Fans.statline['batters']['Position'] == 'OF'].head()

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,AB,Position,Rank,Name,Drafted
playerid,,,,,,,,,,,,,,,
10155,121,82,29,5,37,126,23,114,0.308,1.053,497,OF,1,Mike Trout,False
6184,104,94,33,2,41,130,5,69,0.312,0.994,545,OF,3,J.D. Martinez,False
15640,124,80,26,2,44,129,6,103,0.283,0.985,538,OF,4,Aaron Judge,False
13611,124,106,46,4,30,110,30,83,0.313,0.953,594,OF,2,Mookie Betts,False
11579,104,75,35,1,37,121,10,112,0.280,0.966,528,OF,10,Bryce Harper,False


In [340]:
from scipy.stats import logistic

class Draft:
    
    def __init__(self, projection, number_teams = 12,
                 batters  = {'C','1B','2B', '3B','SS','OF','UTIL'},
                 pitchers = {'SP','RP','P'},
                 number_positions = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5}, 
                 batter_statline  = {'R':0,'1B':0,'2B':0, '3B':0,'HR':0,'RBI':0,'SB':0,'BB':0,'AVG':0,'OPS':0}, 
                 pitcher_statline = {'W':0, 'L':0,'CG':0,'SHO':0,'SV':0,'BB':0,'SO':0,'ERA':0,'WHIP':0,'BSV':0} ):
        
        self.projection = projection
        self.batters  = batters
        self.pitchers = pitchers
        self.open_positions = number_positions
        self.batter_statline = batter_statline
        self.pitcher_statline = pitcher_statline        
        self.number_teams = number_teams

        bat_cols = ['Name']
        for ibat in self.batter_statline:
            bat_cols.append(ibat)
        print(bat_cols)
        bat_df = pd.DataFrame(columns = bat_cols)

        pit_cols = ['Name']
        for ipit in self.pitcher_statline:
            pit_cols.append(ipit)
        print(pit_cols)
        pit_df = pd.DataFrame(columns = pit_cols)
        
        self.bat_df = bat_df
        self.pit_df = pit_df
    
    def draft_team(self, position = 'Best', show_top_N = 12):
        #batter_categories  = ['R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS']
        #pitcher_categories = ['W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV' ]
        self.draftees  = pd.DataFrame()
        for psn in self.batters:
            #print(psn)
            if psn != 'UTIL':
                ind_position_df = (self.projection.statline['batters']['Position']==psn) & (self.projection.statline['batters']['Drafted']=='False')
                nlst = min(len(np.unique(self.projection.statline['batters'][ind_position_df]['Name'])), self.number_teams * self.open_positions[psn] * 2)
                ranked_position_df = self.projection.statline['batters'][ind_position_df].sort_values('Rank')[0:nlst]
                ranked_stat = np.zeros([len(self.batter_statline),nlst])

                for i in range(len(self.batter_statline)):
                    cat = self.batter_statline.keys()[i]
                    
                    if cat in ['HR', 'R', 'RBI', '2B', 'SB', '1B', '3B', 'BB']:
                        ind = np.argsort(ranked_position_df[cat])
                        ranked_stat[i, ind[::-1]]= np.arange(len(ind),0,-1)
                    elif cat in ['AVG', 'OPS']:
                        #weighted_ranked_position_df = ranked_position_df[cat] * (162. * 4.) / ranked_position_df['AB']
                        weighted_ranked_position_df = ranked_position_df[cat] / 2.0 * (1 + logreg(ranked_position_df['AB'], mu = 162.) )
                        ind = np.argsort(weighted_ranked_position_df)
                        ranked_stat[i, ind[::-1]]= np.arange(len(ind),0,-1)
                position_rank = np.sum(ranked_stat.T, axis=1)
                #print(position_rank)

                sorted_rank = np.sort(position_rank)
                sorted_arg = np.argsort(position_rank)
                if show_top_N > 0:
                    for i in (1 + np.arange(min(nlst, show_top_N))): # (1 + np.arange(nlst - 1)):
                        print(psn,ranked_position_df.Name.values[sorted_arg[-i]], sorted_rank[-i])
                
                df = pd.DataFrame([psn,ranked_position_df.Name.values[np.argsort(position_rank)[-1]], np.sort(position_rank)[-1]]) #, columns = ['Position','Name','Score'])
                if self.draftees.empty:
                    self.draftees = df.T
                else:
                    self.draftees = pd.concat([self.draftees, df.T])
                
        for psn in self.pitchers:
            if psn != 'P':
                ind_position_df = (self.projection.statline['pitchers']['Position']==psn) & (self.projection.statline['pitchers']['Drafted']=='False')
                nlst = min(len(np.unique(self.projection.statline['pitchers'][ind_position_df]['Name'])), self.number_teams * self.open_positions[psn] * 2)
                ranked_position_df = self.projection.statline['pitchers'][ind_position_df].sort_values('Rank')[0:nlst]
                ranked_stat = np.zeros([len(self.batter_statline),nlst])

                for i in range(len(self.pitcher_statline)):
                    cat = self.pitcher_statline.keys()[i]
                    
                    if cat in ['SO', 'W', 'SV', 'CG', 'SHO']:
                        ind = np.argsort(ranked_position_df[cat])
                        ranked_stat[i, ind[::-1]]= np.arange(len(ind),0,-1)
                    elif cat in ['L', 'BB', 'BSV']:
                        ind = np.argsort(ranked_position_df[cat])
                        ranked_stat[i, ind]= np.arange(len(ind),0,-1)
                    elif cat in ['WHIP', 'ERA']:                        
                        #weighted_ranked_position_df = ranked_position_df[cat] * (1250. * 9.) / ranked_position_df['IP']
                        if psn == 'SP':
                            mu = 120.
                        else:
                            mu = 50.
                        weighted_ranked_position_df = ranked_position_df[cat] * 2.0 / (1 + logreg(ranked_position_df['IP'], mu = mu) )
                        ind = np.argsort(weighted_ranked_position_df)
                        ranked_stat[i, ind]= np.arange(len(ind),0,-1)                        
                position_rank = np.sum(ranked_stat.T, axis=1)
                if show_top_N > 0:
                    for i in (1 + np.arange(min(nlst, show_top_N))): 
                        print(psn,ranked_position_df.Name.values[np.argsort(position_rank)[-i]], np.sort(position_rank)[-i])
                
                df = pd.DataFrame([psn,ranked_position_df.Name.values[np.argsort(position_rank)[-1]], np.sort(position_rank)[-1]])
                self.draftees = pd.concat([self.draftees, df.T])
        self.draftees.columns = ['Position','Name','Score']
        #self.draftees.index_col = ['Position']
        #print(np.shape(self.draftees))

    def roster_spot_selection(self, position = 'Best'):
        irank = np.argsort(self.draftees['Score'])[::-1]
        for idraft in irank:
            pos = self.draftees.Position.iloc[idraft]
            print(pos)
            if self.open_positions[pos] > 0: # | self.open_positions['BN'] > 0:
                self.fill_roster_spot(self.draftees.iloc[idraft])
                break
                
    def fill_roster_spot(self, pick):
        pos = pick['Position']
        self.open_positions[pos] = self.open_positions[pos] - 1
        if pos in ['P', 'SP', 'RP']:
            inm = np.where(self.projection.pitchers_stats.Name == pick['Name'])
            statline = self.projection.pitchers_stats.iloc[inm][['Name','SO', 'BB', 'W', 'SV', 'ERA', 'BSV', 'CG', 'L', 'WHIP', 'SHO']]
            self.pit_df = pd.concat([self.pit_df, statline])
        else:
            inm = np.where(self.projection.hitters_stats.Name == pick['Name'])
            statline = self.projection.hitters_stats.iloc[inm][['Name', 'HR', 'R', 'RBI', '2B', 'SB', '1B', '3B', 'BB', 'AVG', 'OPS']]
            self.bat_df = pd.concat([self.bat_df, statline])
        print(self.bat_df)

def logreg(x, mu = 0):
    return 1.0 / (1 + np.exp(-1 * (x - mu)))

In [341]:
number_teams = 12
ZiPS_Dft = Draft(ZiPS,number_teams = number_teams)
Steamer_Dft = Draft(Steamer,number_teams = number_teams)
Fans_Dft = Draft(Fans,number_teams = number_teams)

['Name', 'HR', 'R', 'RBI', '2B', 'SB', '1B', '3B', 'BB', 'AVG', 'OPS']
['Name', 'SO', 'BB', 'W', 'SV', 'ERA', 'BSV', 'CG', 'L', 'WHIP', 'SHO']
['Name', 'HR', 'R', 'RBI', '2B', 'SB', '1B', '3B', 'BB', 'AVG', 'OPS']
['Name', 'SO', 'BB', 'W', 'SV', 'ERA', 'BSV', 'CG', 'L', 'WHIP', 'SHO']
['Name', 'HR', 'R', 'RBI', '2B', 'SB', '1B', '3B', 'BB', 'AVG', 'OPS']
['Name', 'SO', 'BB', 'W', 'SV', 'ERA', 'BSV', 'CG', 'L', 'WHIP', 'SHO']


In [342]:
Fans_Dft.draft_team(show_top_N = 0)

In [343]:
Fans_Dft.roster_spot_selection()

OF
               Name  HR    R  RBI  2B  SB   1B 3B  BB    AVG    OPS
13611  Mookie Betts  30  124  110  46  30  106  4  83  0.313  0.953


In [347]:
print(Fans_Dft.open_positions)
print(Fans_Dft.bat_df)
print(Fans_Dft.pit_df)

{'C': 1, 'BN': 5, 'UTIL': 1, 'P': 3, '1B': 1, '2B': 1, '3B': 1, 'RP': 2, 'SS': 1, 'OF': 2, 'SP': 2}
               Name  HR    R  RBI  2B  SB   1B 3B  BB    AVG    OPS
13611  Mookie Betts  30  124  110  46  30  106  4  83  0.313  0.953
Empty DataFrame
Columns: [Name, SO, BB, W, SV, ERA, BSV, CG, L, WHIP, SHO]
Index: []


In [149]:
Fans_Dft.open_positions['C']
idf = np.where(Fans_Dft.bat_df['Position'] == 'C')[0][0]
#Fans_Dft.bat_df.iloc[idf] = 
imb = np.where(Steamer.hitters_stats.Name == 'Mookie Betts')
Steamer.hitters_stats.iloc[imb][['Name', 'HR', 'R', 'RBI', '2B', 'SB', '1B', '3B', 'BB', 'AVG', 'OPS']]

,Name,HR,R,RBI,2B,SB,1B,3B,BB,AVG,OPS
playerid,,,,,,,,,,,
13611,Mookie Betts,32,112,102,42,26,105,4,77,0.305,0.936


array([2, 6, 5, 3, 4, 1, 0, 7])

In [69]:
ZiPS_Dft.draft_team(show_top_N = 5)

('C', 'J.T. Realmuto', 224.0)
('C', 'Willson Contreras', 202.0)
('C', 'Buster Posey', 187.0)
('C', 'Yadier Molina', 167.0)
('C', 'Yasmani Grandal', 164.0)
('SS', 'Francisco Lindor', 210.0)
('SS', 'Trea Turner', 191.0)
('SS', 'Manny Machado', 189.0)
('SS', 'Xander Bogaerts', 182.0)
('SS', 'Trevor Story', 166.0)
('OF', 'Mookie Betts', 651.0)
('OF', 'Christian Yelich', 649.0)
('OF', 'Mike Trout', 622.0)
('OF', 'Juan Soto', 621.0)
('OF', 'Charlie Blackmon', 615.0)
('1B', 'Freddie Freeman', 219.0)
('1B', 'Cody Bellinger', 207.0)
('1B', 'Anthony Rizzo', 204.0)
('1B', 'Paul Goldschmidt', 201.0)
('1B', 'Joey Votto', 163.0)
('2B', 'Jose Altuve', 199.0)
('2B', 'Ozzie Albies', 188.0)
('2B', 'Javier Baez', 173.0)
('2B', 'Whit Merrifield', 173.0)
('2B', 'Brian Dozier', 164.0)
('3B', 'Jose Ramirez', 220.0)
('3B', 'Nolan Arenado', 201.0)
('3B', 'Alex Bregman', 201.0)
('3B', 'Kris Bryant', 185.0)
('3B', 'Anthony Rendon', 164.0)
('SP', 'Chris Sale', 378.0)
('SP', 'Clayton Kershaw', 352.0)
('SP', 'Corey

In [70]:
Steamer_Dft.draft_team()

('C', 'J.T. Realmuto', 220.0)
('C', 'Buster Posey', 205.0)
('C', 'Yadier Molina', 195.0)
('C', 'Gary Sanchez', 192.0)
('C', 'Willson Contreras', 185.0)
('C', 'Francisco Cervelli', 175.0)
('C', 'Yasmani Grandal', 169.0)
('C', 'Tucker Barnhart', 155.0)
('C', 'Danny Jansen', 137.0)
('C', 'Welington Castillo', 124.0)
('C', 'Wilson Ramos', 123.0)
('C', 'Jorge Alfaro', 121.0)
('SS', 'Trevor Story', 181.0)
('SS', 'Francisco Lindor', 177.0)
('SS', 'Trea Turner', 174.0)
('SS', 'Manny Machado', 171.0)
('SS', 'Carlos Correa', 171.0)
('SS', 'Xander Bogaerts', 169.0)
('SS', 'Corey Seager', 161.0)
('SS', 'Javier Baez', 150.0)
('SS', 'Elvis Andrus', 139.0)
('SS', 'Jorge Polanco', 130.0)
('SS', 'Jean Segura', 129.0)
('SS', 'Jose Peraza', 125.0)
('OF', 'Mookie Betts', 668.0)
('OF', 'Mike Trout', 647.0)
('OF', 'Christian Yelich', 604.0)
('OF', 'Charlie Blackmon', 598.0)
('OF', 'Juan Soto', 596.0)
('OF', 'Andrew Benintendi', 590.0)
('OF', 'J.D. Martinez', 587.0)
('OF', 'Ronald Acuna Jr.', 583.0)
('OF', '

In [11]:
number_positions = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5}

In [14]:
statline = {}
statline['batter_statline'] = {'R':0,'1B':0,'2B':0,'3B':0,'HR':0,'RBI':0,'SB':0,'BB':0,'AVG':0,'OPS':0}
statline['pitcher_statline'] = {'W':0,'L':0,'CG':0,'SHO':0,'SV':0,'BB':0,'K':0,'ERA':0,'WHIP':0,'BSV':0}

In [12]:
print(number_positions.keys())
print(number_positions.values())
np.argsort(number_positions.values())[::-1] 

['C', 'BN', 'UTIL', 'P', '1B', '2B', '3B', 'RP', 'SS', 'OF', 'SP']
[1, 5, 1, 3, 1, 1, 1, 2, 1, 3, 2]


array([ 1,  9,  3, 10,  7,  8,  6,  5,  4,  2,  0])